In [8]:
import os
# import json
import math
import torch
from itertools import product
from dataclasses import dataclass, field

from gamesopt.attacks import Attack
from gamesopt.optimizer import Optimizer
from gamesopt.aggregator import Aggregator
from gamesopt.games import Game, create_matrix, create_bias
from gamesopt.train_distributed import train

# %matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

In [10]:
# class Config(BaseConfig):
@dataclass
class Config():
    n_iter: int = 6001

    n_peers: int = 20
    n_byzan: int = 4

    # game: Game = Game.Quadratic
    game: Game = field(default_factory=lambda: Game.Quadratic)
    num_samples: int = 1000
    dim: int = 25
    with_bias: bool = True
    mu: float = 1e-1
    ell: float = 1e3

    optimizer: Optimizer = None
    batch_size: int = None
    lr: float = 1/2/ell/15
    alpha: float = None
    sigmaC: float = None

    attack: Attack = None
    n_attacking: int = None
    attack_param: float = None

    use_bucketing: bool = None
    bucketing_s: int = 2

    aggregator: Aggregator = None
    aggregator_param_a: int = 10  # trimmed_mean_b=10; krum_m=2; ,clipping_n_iter=3 clipping_tau=10
    aggregator_param_b: float = 0.1 # rfa_T=10,rfa_nu=0.1    , 0.1)  # rfa_T=10,rfa_nu=0.1

In [11]:
@dataclass
class Data():
    def __init__(self, config):
        self.matrix = create_matrix(config.dim, config.num_samples,
                                    config.mu, config.ell,
                                    with_bias=config.with_bias)
        self.bias = create_bias(config.dim, config.num_samples,
                                with_bias=config.with_bias)
        self.true = torch.linalg.solve(self.matrix.mean(dim=0),
                                       -self.bias.mean(dim=0))
        self.players = self.true + .1/math.sqrt(2 * config.dim)

In [12]:
base_config = Config()
data = Data(base_config)
# base_config_items = set(base_config.__dict__.items())

In [13]:
for at, ap, bs in product([Attack.RN],
                          [1e1, 1e2, 1e3, 1e4],
                          [1, 10, 100]):

    # SGDA
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.SGDA
    config.aggregator = Aggregator.Mean
    config.n_attacking = config.n_byzan
    # config_items = set(config.__dict__.items())
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    # json.dumps(dict(sorted(config_items - base_config_items)))
    train(config, data)

    # SGDARA
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.SGDARA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # MSGDARA
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.MSGDARA
    config.alpha = 0.1
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA M'
    os.environ['MLFLOW_RUN_NAME'] = 'MSGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGRA
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.SEGRA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGRA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # RDEG
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.RDEG
    config.aggregator = Aggregator.UnivariateTM
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'RDEG'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SGDACC
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter * 2 - 1
    config.optimizer = Optimizer.SGDACC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDACC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGCC
    config = Config(attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter + 1
    config.optimizer = Optimizer.SEGCC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGCC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)


Trying port 41285
Trying port 45128
Trying port 37206
Trying port 42558
Trying port 18610
Trying port 6318
Trying port 37410
Trying port 31723
Trying port 44992
Trying port 49020
Trying port 15290
Trying port 36402
Trying port 35713
Trying port 11940
Trying port 38065
Trying port 6609
Trying port 12796
Trying port 29629
Trying port 6678
Trying port 31684
Trying port 15813
Trying port 48615
Trying port 25758
Trying port 38306
Trying port 4235
Trying port 18370
Trying port 7919
Trying port 30883
Trying port 12301
Trying port 32538
Trying port 46324
Trying port 13519
Trying port 38383
Trying port 41494
Trying port 7184
Trying port 28230
Trying port 49139
Trying port 39071
Trying port 42688
Trying port 35243
Trying port 2929
Trying port 43355
Trying port 46827
Trying port 13976
Trying port 32189
Trying port 29809
Trying port 47134
Trying port 34207
Trying port 20962
Trying port 40034
Trying port 28334
Trying port 12183
Trying port 45453
Trying port 26878
Trying port 1346
Trying port 23831
